In [2]:
import pandas as pd
import spotipy
import spotipy.util as util
import json
import numpy as np
from tqdm import tqdm
from google.cloud import language_v1
from google.cloud.language_v1 import enums
import six
import os

In [3]:
key_file = 'assets/keys.json'
keys = json.load(open(key_file))

SPOTIPY_CLIENT_ID= keys["client_id"]
SPOTIPY_CLIENT_SECRET= keys["client_secret"]
SPOTIPY_REDIRECT_URI= keys["redirect_url"]

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/carlosortega/Documents/GitHub/SUSA-Research-MusicMoods/assets/google_keys.json"

In [4]:
token = util.prompt_for_user_token("1274418332", 
                                   scope = "user-library-read", 
                                   client_id=SPOTIPY_CLIENT_ID, 
                                   client_secret=SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri = SPOTIPY_REDIRECT_URI)
spotify = spotipy.Spotify(auth=token)

In [5]:
billboard = pd.read_csv("data/billboard_lyrics_1964-2015.csv")
billboard.head()

,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0


In [6]:
billboard['Query'] = 'artist: ' + billboard['Artist'] + ', track: ' + billboard['Song']

In [7]:
sixties = billboard[billboard['Year'] < 1970]
seventies = billboard[(billboard['Year'] >= 1970) & (billboard['Year'] < 1980)]
eighties = billboard[(billboard['Year'] >= 1980) & (billboard['Year'] < 1990)]
nineties = billboard[(billboard['Year'] >= 1990) & (billboard['Year'] < 2000)]
thousands = billboard[(billboard['Year'] >= 2000) & (billboard['Year'] < 2010)]
twenty_tens = billboard[(billboard['Year'] >= 2010) & (billboard['Year'] < 2020)]

In [8]:
#song_id = spotify.search("artist: kesha, track: tik tok")['tracks']['items'][0]['id']
#valence = spotify.audio_features(song_id)[0]['valence']

In [9]:
#valences = []
#for index, row in billboard.iterrows():
#    try:
#        curr_song = spotify.search(row['Query'])['tracks']['items'][0]['id']
#        valence = spotify.audio_features(curr_song)[0]['valence']
#        valences.append(valence)
#    except:
#        valences.append("ERROR. INDEX: " + str(index))
#    if index % 10 == 0:
#        print(index)

#billboard['Valence'] = valences

In [10]:
#billboard.to_csv("data/billboard_with_data.csv", index=False)
#billboard.head()

In [11]:
billboard_updated = pd.read_csv("data/billboard_with_data.csv")
billboard_updated['Lyrics'].replace('  ', np.nan, inplace=True)
billboard_updated.dropna(axis=0,inplace=True)
billboard_updated.drop('Source', axis = 1, inplace=True)
billboard_updated.head()

,Rank,Song,Artist,Year,Lyrics,Query,Valence
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,"artist: sam the sham and the pharaohs, track: ...",0.558
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,"artist: four tops, track: i cant help myself s...",0.964
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,"artist: we five, track: you were on my mind",0.7
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,"artist: the righteous brothers, track: youve l...",0.376
5,6,downtown,petula clark,1965,when youre alone and life is making you lonel...,"artist: petula clark, track: downtown",0.545


In [12]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)
implicit()

[]


In [13]:
def analyze_sentiment(content):

    client = language_v1.LanguageServiceClient()

    if isinstance(content, six.binary_type):
        content = content.decode('utf-8')

    type_ = enums.Document.Type.PLAIN_TEXT
    document = {'type': type_, 'content': content}

    response = client.analyze_sentiment(document)
    sentiment = response.document_sentiment
    return sentiment.score

In [17]:
help_ = billboard_updated[billboard_updated['Song'] == 'help']["Lyrics"].values[0]
analyze_sentiment(help_)

RetryError: Deadline of 600.0s exceeded while calling functools.partial(<function _wrap_unary_errors.<locals>.error_remapped_callable at 0x116c3f1e0>, document {
  type: PLAIN_TEXT
  content: "help i need somebody help not just anybody help you know i need someone help when i was younger so much younger than today i never needed anybodys help in any way but now these days are gone im not so selfassured now i find ive changed my mind and opened up the doors help me if you can im feeling down and i do appreciate you being round help me get my feet back on the ground wont you please please help me and now my life has changed in oh so many ways my independence seems to vanish in the haze but every now and then i feel so insecure i know that i just need you like ive never done before help me if you can im feeling down and i do appreciate you being round help me get my feet back on the ground wont you please please help me when i was younger so much younger than today i never needed anybodys

In [16]:
#billboard_updated['Sentiment'] = billboard_updated['Lyrics'].apply(analyze_sentiment)
sentiment = []
for index in tqdm(range(billboard_updated.shape[0])):
    row = billboard_updated.iloc[index]
    sentiment.append(analyze_sentiment(row['Lyrics']))


  0%|          | 1/4868 [00:01<2:05:36,  1.55s/it]

KeyboardInterrupt: 

Exception ignored in: 'grpc._cython.cygrpc._next'
KeyboardInterrupt


KeyError: None

In [ ]:
billboard_updated.head()